In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import glob
import json
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.preprocessing import image # type: ignore
from tensorflow.keras.applications.resnet50 import preprocess_input # type: ignore

# Configuration
IMG_SIZE = (224, 224)
NUM_CENTROIDS_PER_PERSON = 16
NUM_PEOPLE = 488

# Load model and data
base_model = load_model('models/resnet50.h5')
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=base_model.get_layer('conv5_block3_out').output)


with open("../models/feature_maps/class_indices.json", "r") as f:
    index_to_class = {v: k for k, v in json.load(f).items()}

def detect_and_crop_face(image):
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 3)
        detected_faces = []
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 1) 
            side_length = max(w, h)  
            half_side = side_length // 2
            center_x = x + w // 2
            center_y = y + h // 2
        
            x1 = max(center_x - half_side, 0)
            y1 = max(center_y - half_side, 0)
            x2 = min(center_x + half_side, image.shape[1])
            y2 = min(center_y + half_side, image.shape[0])
            
            face_img = image[y1:y2, x1:x2]
            detected_faces.append(face_img)  
        return detected_faces

def preprocess_image(self, img_path):
      img = cv2.imread(img_path)
      if img is None:
            raise ValueError(f"Error: Could not read image {img_path}.")
      detected_faces = detect_and_crop_face(img)
      if not detected_faces:
            print("Error: No faces detected.")
            return None, None
      img = detected_faces[0]
      img = cv2.resize(img, self.img_size)
      img_array = image.img_to_array(img)
      return preprocess_input(np.expand_dims(img_array, axis=0)), img

def load_feature_maps(self, data_dir="../models/data"):
      feature_maps = []
      image_paths = []
      for class_dir in os.listdir(data_dir):
            if os.path.isdir(os.path.join(data_dir, class_dir)):
                  for img_file in glob.glob(os.path.join(data_dir, class_dir, "*.jpeg")):
                        img_array, _ = self.preprocess_image(img_file)
                        feature_map = self.feature_extractor.predict(img_array)
                        feature_maps.append(feature_map.flatten())
                        image_paths.append(img_file)
      return np.array(feature_maps), image_paths


def find_similar_images(self, input_feature, num_similar=3):
      distances = np.linalg.norm(self.feature_maps - input_feature, axis=1)
      closest_indices = np.argsort(distances)[:num_similar]
      return [self.image_paths[i] for i in closest_indices]

def visualize_similar_images(self, img_path, num_similar=3):
      input_feature, _ = self.preprocess_image(img_path)
      input_feature = self.feature_extractor.predict(input_feature).flatten()
      similar_images = self.find_similar_images(input_feature, num_similar)
      
      return similar_images


image_paths = [
    'path_to_image_1.jpg',
    'path_to_image_2.jpg',
    'path_to_image_3.jpg',
    'path_to_image_4.jpg',
    'path_to_image_5.jpg'
]

# Hàm để kiểm tra
def test_visualize_similar_images(image_paths):
    for img_path in image_paths:
        try:
            similar_images = visualize_similar_images(img_path, num_similar=3)
            print(f"Similar images for {img_path}:")
            for similar_img in similar_images:
                print(f" - {similar_img}")
        except ValueError as e:
            print(f"Error for {img_path}: {e}")

# Gọi hàm kiểm tra
test_visualize_similar_images(image_paths)